In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import pandas as pd

import sklearn
from sklearn import svm, preprocessing 
import seaborn as sns

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import os
import plotly.graph_objs as go

In [2]:
df=pd.read_csv('epoch10_XAUTRY.csv')

In [3]:
# replace , with . in csv file
df = df.replace(',','', regex=True)
df = df.astype(float)

In [4]:
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df)
df_normalized = pd.DataFrame(np_scaled)
df=df_normalized
df

,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.025673,0.013997,0.007075,0.011427,0.023281,0.020123
1,0.013988,0.004604,0.018164,0.020937,0.029336,0.015168,0.007234,0.011427,0.025617,0.021221,0.000000
2,0.018845,0.024083,0.038720,0.049659,0.018665,0.007839,0.011683,0.025617,0.023350,0.000000,0.021314
3,0.039397,0.046129,0.066921,0.039211,0.011220,0.012662,0.026192,0.023350,0.000000,0.022476,0.035701
4,0.062657,0.076372,0.056663,0.031922,0.016119,0.028385,0.023874,0.000000,0.024731,0.037649,0.056641
...,...,...,...,...,...,...,...,...,...,...,...
78,0.948814,0.880890,0.828604,0.847400,0.855116,0.907596,0.905356,0.853057,0.849020,0.820195,0.842893
79,0.943393,0.893234,0.850171,0.858150,0.925263,0.981164,0.872206,0.849020,0.902479,0.888872,0.861813
80,0.956418,0.916365,0.860726,0.926828,1.000000,0.945238,0.868078,0.902479,0.978046,0.908825,0.860532
81,0.980822,0.927684,0.928157,1.000000,0.963503,0.940764,0.922737,0.978046,1.000000,0.907474,0.948272


In [5]:
df.isnull().values.any()

False

In [6]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,10
count,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000
mean,0.638200,0.603835,0.570854,0.574755,0.577473,0.577328,0.544485,0.544414,0.556359,0.517556,0.502566
std,0.311065,0.290420,0.267757,0.268461,0.269683,0.262087,0.240517,0.233613,0.231137,0.209695,0.199707
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.526603,0.497508,0.461344,0.453219,0.465219,0.495045,0.475664,0.485737,0.504095,0.461732,0.441474
50%,0.741583,0.691179,0.643050,0.641774,0.647131,0.635315,0.597406,0.584360,0.588389,0.544308,0.517686
75%,0.863054,0.807772,0.751480,0.757731,0.762741,0.750422,0.696484,0.684514,0.692931,0.641203,0.614812
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
from sklearn.model_selection import train_test_split
X= df.iloc[:,:-1]
y=df.iloc[:,-1]

In [8]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection  import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error

In [9]:
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size = 0.3)

In [10]:
#Taking odd integers as K vales so that majority rule can be applied easily. 
neighbors = np.arange(1, 20, 2)
scores = []

In [11]:
for k in neighbors:   # running for different K values to know which yields the max accuracy. 
    clf = KNeighborsRegressor(n_neighbors = k,  weights = 'distance', p=1)
    clf.fit(X_tr, y_tr)
    score = cross_val_score(clf, X_tr, y_tr, cv = 10)
    scores.append(score.mean())

In [12]:
mse = [1-x for x in scores]

In [13]:
trace0 = go.Scatter(
    y = mse,
    x = neighbors,
    mode = 'lines+markers', 
    marker = dict(
        color = 'rgb(150, 10, 10)'
    )
)
layout = go.Layout(
    title = '', 
    xaxis = dict(
        title = 'K value', 
        tickmode = 'linear'
    ),
    yaxis = dict(
        title = 'CV Error',
    )
)
fig = go.Figure(data = [trace0], layout = layout)
iplot(fig, filename='basic-line')

In [14]:
optimal_k = neighbors[mse.index(min(mse))]
print("Optimal K: ", optimal_k)

Optimal K:  7


In [15]:
# Training the model on Optimal K.
clf_optimal = KNeighborsRegressor(n_neighbors = optimal_k)
clf_optimal.fit(X_tr, y_tr)
y_pred = clf_optimal.predict(X_test)
acc = clf_optimal.score(X_test, y_test)
print("Accuracy: ", acc*100)
print("RMS Error: ", mean_squared_error(y_test, y_pred))

Accuracy:  83.35612504315546
RMS Error:  0.008050925003037767
